# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokurdakh,70.6333,147.9167,-40.14,100,91,5.91,RU,1674330771
1,1,meulaboh,4.1363,96.1285,73.51,90,99,3.51,ID,1674330771
2,2,bambous virieux,-20.3428,57.7575,77.25,88,40,3.44,MU,1674330772
3,3,qaanaaq,77.4840,-69.3632,-29.90,61,39,7.47,GL,1674330772
4,4,halifax,44.6453,-63.5724,29.50,80,94,10.00,CA,1674330600


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
filter1_city_df = city_data_df.loc[city_data_df["Max Temp"] < 81, :]
filter2_city_df = filter1_city_df.loc[filter1_city_df["Max Temp"] > 70, :]
filter3_city_df = filter2_city_df.loc[filter2_city_df["Wind Speed"] < 10, :] 
filter4_city_df = filter3_city_df.loc[filter3_city_df["Cloudiness"] == 0, :] 

# Drop any rows with null values
# Not needed, no null values as shown below. 

# Display sample data
filter4_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,cape town,-33.9258,18.4232,72.37,62,0,3.44,ZA,1674330780
39,39,goure,13.9835,10.2704,71.40,13,0,7.63,NE,1674330784
100,100,pisco,-13.7000,-76.2167,77.05,69,0,9.22,PE,1674330804
116,116,cabo san lucas,22.8909,-109.9124,76.60,59,0,5.99,MX,1674330789
189,189,saint-pierre,-21.3393,55.4781,78.48,83,0,5.75,RE,1674330835
337,337,tessalit,20.1986,1.0114,73.96,12,0,6.91,ML,1674330889
339,339,kahului,20.8947,-156.4700,78.84,72,0,8.01,US,1674330890
383,383,mandurah,-32.5500,115.7000,78.06,42,0,7.90,AU,1674330906
407,407,ullal,12.8000,74.8500,76.15,83,0,3.44,IN,1674330914
454,454,lima,-12.0432,-77.0282,73.65,69,0,9.22,PE,1674330815


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filter4_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,cape town,ZA,-33.9258,18.4232,62,
39,goure,NE,13.9835,10.2704,13,
100,pisco,PE,-13.7000,-76.2167,69,
116,cabo san lucas,MX,22.8909,-109.9124,59,
189,saint-pierre,RE,-21.3393,55.4781,83,
337,tessalit,ML,20.1986,1.0114,12,
339,kahului,US,20.8947,-156.4700,72,
383,mandurah,AU,-32.5500,115.7000,42,
407,ullal,IN,12.8000,74.8500,83,
454,lima,PE,-12.0432,-77.0282,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":f"accommodation.hotel", 
          "limit":20,
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


TypeError: 'method' object is not subscriptable

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 400,
    size = 250, 
    color = "Country"

)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country)